In [ ]:
import os
from dotenv import load_dotenv
from llama_index import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    ServiceContext,
    OpenAIEmbedding,
    OpenAI,
    ReActAgent,
    FunctionTool,
    ToolMetadata,
    QueryEngineTool,
)

# Load environment variables
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

# Step 1: Load your wiki documents from a local folder
documents = SimpleDirectoryReader("./wiki_data").load_data()

# Step 2: Set up LLM and embedding model
llm = OpenAI(model="gpt-4-turbo", api_key=openai_api_key)
embed_model = OpenAIEmbedding(model="text-embedding-ada-002", api_key=openai_api_key)

service_context = ServiceContext.from_defaults(llm=llm, embed_model=embed_model)

# Step 3: Build the vector index
index = VectorStoreIndex.from_documents(documents, service_context=service_context)

# Step 4: Create a query engine tool
query_engine_tool = QueryEngineTool.from_defaults(
    query_engine=index.as_query_engine(similarity_top_k=5),
    metadata=ToolMetadata(name="wiki_query_tool", description="Query engine for wiki documents"),
)

# Step 5: Initialize the ReAct agent with the query engine tool
agent = ReActAgent.from_tools([query_engine_tool], llm=llm, verbose=True)

# Step 6: Interact with the AI Wiki Agent
while True:
    query = input("Ask the Wiki Agent: ")
    if query.lower() in ["exit", "quit"]:
        break
    response = agent.chat(query)
    print("\nAnswer:", response)


ImportError: cannot import name 'SimpleDirectoryReader' from 'llama_index' (unknown location)